In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import label_binarize

def geradataset(tamanho=20, centro=2):
    X, y = make_blobs(n_samples=tamanho, centers=centro, center_box=(0,1.0), cluster_std=0.02)
    return X, y

def plotadataset(X, y):
    plt.xlabel("X1")
    plt.ylabel("X2")
    for k in set(y):
        plt.plot(X[:,0][y==k],X[:,1][y==k],"o", alpha=0.3)


def plotahiperplano(vetor, bias=0, xmin=0, xmax=1):
    xs = np.linspace(xmin, xmax, num=2)
    ys = (-vetor[0] / vetor[1]) * xs - bias / vetor[1]
    plt.plot(xs, ys)
    

X, y = geradataset(100, 3)
plotadataset(X, y)
plt.show()
y

## Função de Custo

In [ ]:
class CustoPerceptron():
    @staticmethod
    def erro(y, ypred):
        return y - ypred
    @staticmethod
    def custo(y, ypred):
        return np.sum(CustoPerceptron.erro(y, ypred)**2)
    @staticmethod
    def gradiente(y, ypred, X):
        return np.matmul(X.T ,CustoPerceptron.erro(y, ypred))
    
class Adaline():
    def __init__(self):
        self.preactivated = True
    @staticmethod
    def erro(y, ypred):
        return y - ypred
    @staticmethod
    def custo(y, ypred):
        return np.sum((1 - Adaline.erro(y, ypred)) ** 2)
    @staticmethod
    def gradiente(y, ypred, X):
        return np.matmul(X.T, Adaline.erro(y, ypred))

## Algoritmos

In [ ]:
class DescidaGradiente():
    def __init__(self, custo=Adaline(), maxiter=1000, alpha=0.005):
        self.custo = custo
        self. maxiter = maxiter
        self.alpha = alpha
    
    def getW(self, X, y, activation=lambda a: a):
        w = np.random.uniform(-1, 1, size=(X.shape[1], y.shape[1]))
        for _ in range(self.maxiter):
            ypred = activation(np.matmul(X, w))
            custo = self.custo.custo(y, ypred)
            if custo == 0:
                break
            w = w + self.alpha * self.custo.gradiente(y, ypred, X)
        return w

class PseudoInversa():
    def __init__(self):
        pass
    def getW(self, X, y):
        pinv = np.linalg.pinv(X)
        w = np.matmul(pinv, y)
        return w

## Perceptron

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class Perceptron(BaseEstimator, ClassifierMixin):
    
    def __init__(self, algoritmo=DescidaGradiente(Adaline())):
        self.w = None
        self.activation = lambda a: (a >= 0)*2-1
        self.threshold = 0
        self.algoritmo = algoritmo

    @staticmethod
    def includebias(X):
        bias = np.ones((X.shape[0],1))
        Xb = np.concatenate((bias, X), axis=1)
        return Xb

    def fit(self, X ,y):
        X = Perceptron.includebias(X)
        self.labels = list(set(y))
        y = label_binarize(y, classes=self.labels)*2 - 1
        if len(self.labels) == 2:
            y = y[:,0:1]
        # Treinamento
        if hasattr(self.algoritmo, 'custo') and not (hasattr(self.algoritmo.custo, 'preactivated') and  self.algoritmo.custo.preactivated):
            self.w  = self.algoritmo.getW(X, y, self.activation)
        else:
            self.w  = self.algoritmo.getW(X, y)
        

    def predict(self, X):
        Xb = Perceptron.includebias(X)
        a = np.matmul(Xb, self.w)
        if self.w.shape[1] > 1:
            idx = np.argmax(a, axis=1)
        else:
            idx = np.array(self.activation(a) > self.threshold, dtype=int)[:,0]
        ypred = np.array([self.labels[i] for i in idx])
        return ypred

In [ ]:
perceptron = Perceptron(DescidaGradiente(Adaline()))
perceptron.fit(X, y)
ypred = perceptron.predict(X)
ypred
print(sum(y == ypred)/len(y))
print(perceptron.w[1:], perceptron.w[0])

plotadataset(X, y)
if len(set(y)) > 2:
    for i in range(len(set(y))):
        plotahiperplano(perceptron.w[1:, i], perceptron.w[0, i], min(X[:,0]), max(X[:,0]))
else:
    plotahiperplano(perceptron.w[1:], perceptron.w[0])

In [ ]:
perceptron = Perceptron(DescidaGradiente(CustoPerceptron()))
perceptron.fit(X, y)
ypred = perceptron.predict(X)
ypred
print(sum(y == ypred)/len(y))
print(perceptron.w[1:], perceptron.w[0])

plotadataset(X, y)
if len(set(y)) > 2:
    for i in range(len(set(y))):
        plotahiperplano(perceptron.w[1:, i], perceptron.w[0, i], min(X[:,0]), max(X[:,0]))
else:
    plotahiperplano(perceptron.w[1:], perceptron.w[0])

In [ ]:
perceptron = Perceptron(PseudoInversa())
perceptron.fit(X, y)
ypred = perceptron.predict(X)
ypred
print(sum(y == ypred)/len(y))
print(perceptron.w[1:], perceptron.w[0])

plotadataset(X, y)
if len(set(y)) > 2:
    for i in range(len(set(y))):
        plotahiperplano(perceptron.w[1:, i], perceptron.w[0, i], min(X[:,0]), max(X[:,0]))
else:
    plotahiperplano(perceptron.w[1:], perceptron.w[0])